In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [4]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [5]:
 x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


In [6]:
from sklearn.model_selection import cross_validate

SEED = 158020 
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print(f'Accuracy com cross validation, 3: [{(media - 2 * desvio_padrao)*100:.2f}, {(media + 2 * desvio_padrao)*100:.2f}]')

Accuracy com cross validation, 3: [74.99, 76.57]


In [7]:
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print(f'Accuracy com cross validation, 3: [{(media - 2 * desvio_padrao)*100:.2f}, {(media + 2 * desvio_padrao)*100:.2f}]') 

Accuracy com cross validation, 3: [75.21, 76.35]


# Aleatoriedade no cross validate

In [8]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print('Accuracy médio %.2f' %(media * 100))
    print(f'Accuracy intervalo: [{(media - 2 * desvio_padrao)*100:.2f}, {(media + 2 * desvio_padrao)*100:.2f}]') 

In [9]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy médio 75.78
Accuracy intervalo: [74.37, 77.19]


In [10]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy médio 75.76
Accuracy intervalo: [73.26, 78.26]


# Simular situação horrível de azar

Pode ser "azar" como pode ser uma proporção de exemplos desbalanceadas entre as classes

In [11]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [12]:
# Rodando sem o Shuffle. Percebe-se que o resultado foi bem abaixo do esperado

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy médio 57.84
Accuracy intervalo: [34.29, 81.39]


In [13]:
# Com o shuffle 

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy médio 75.78
Accuracy intervalo: [72.88, 78.68]


In [14]:
# Utilizando agora o StratifiedKFold, que faz a estratificacao das bases na
# divisao (mesma proporcao de determinada caracteristica em cada uma delas)
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

Accuracy médio 75.79
Accuracy intervalo: [73.68, 77.90]


# Dados agrupáveis

In [15]:
# Criaremos uma coluna com o modelo do carro. 
# Sabemos que o modelo está fortemente atrealado com a idade do modelo

dados.idade_do_modelo

0       18
1       20
2       12
3        3
4        4
        ..
9995    12
9996    16
9997     4
9998     7
9999    19
Name: idade_do_modelo, Length: 10000, dtype: int64

In [16]:
# Trazendo um pouco de aleatoriedade e atribuindo o modelo como numeros

np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=dados.shape[0])
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [17]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1], dtype=int64)

In [18]:
dados.modelo_aleatorio.min()

-1

In [19]:
# Como tem modelo -1, o que não é desejável, soma-se um valor a todos os numeros da coluna
# Somou-se o seu valor absoluto para chegar a 1 e mais 1 para o modelo inicial ser 1 e nao 0

np.random.seed(SEED)
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) +1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [20]:
# Realizaando o procedimento agora da forma final:

np.random.seed(SEED)
dados.drop(['modelo_aleatorio'], 1, inplace = True)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=dados.shape[0])
dados.modelo = dados.modelo + abs(dados.modelo.min()) +1
dados.head()

C:\Users\thall\AppData\Local\Temp/ipykernel_34688/1758349775.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dados.drop(['modelo_aleatorio'], 1, inplace = True)


,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [21]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

In [22]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

In [23]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results)

Accuracy médio 75.80
Accuracy intervalo: [72.00, 79.60]


# Cross validation com StandartScaler

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi de %.2f %%" % acuracia)

A acurácia foi de 77.48 %


In [25]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio 77.27
Accuracy intervalo: [74.35, 80.20]


In [26]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [27]:
x_azar_escalado

array([[ 0.3636103 , -0.39591706,  0.24651152],
       [ 0.79214444, -0.18257669,  0.03346607],
       [ 0.72310419,  1.09746554,  1.66435195],
       ...,
       [ 0.17822399, -0.60925743, -0.73405662],
       [ 0.2248779 ,  0.45744443, -0.39741686],
       [-1.34266351,  0.88412517,  1.52099053]])

In [28]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio 76.70
Accuracy intervalo: [74.30, 79.10]


In [29]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio 76.70
Accuracy intervalo: [74.30, 79.10]
